In [1]:
#pip install transformers torch scikit-learn joblib requests pandas numpy
import os
import torch
import joblib
import requests
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load Model and Tokenizer from Hugging Face
model_name = "AZ0202/bert-genre-classification"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# ✅ Move Model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Download test dataset
TEST_CSV_URL = "https://raw.githubusercontent.com/rixprakash/Jarheads/main/Project%201/DATA/test_data.csv"
TEST_CSV_PATH = "test_data.csv"

response = requests.get(TEST_CSV_URL)
with open(TEST_CSV_PATH, "wb") as f:
    f.write(response.content)

# Load Test Data
df_test = pd.read_csv(TEST_CSV_PATH)
print(f"✅ Loaded Test Data! Shape: {df_test.shape}")

# Download and Load Label Encoder
LABEL_ENCODER_URL = "https://raw.githubusercontent.com/rixprakash/Jarheads/main/Project%201/OUTPUT/label_encoder.pkl"
LABEL_ENCODER_PATH = "label_encoder.pkl"

response = requests.get(LABEL_ENCODER_URL)
with open(LABEL_ENCODER_PATH, "wb") as f:
    f.write(response.content)

label_encoder = joblib.load(LABEL_ENCODER_PATH)

# Function to Predict Genre for One Sample
def predict_genre(lyrics):
    inputs = tokenizer(lyrics, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to GPU if available

    with torch.no_grad():
        outputs = model(**inputs)

    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return predicted_class  # Returns index of predicted genre

# Convert All Test Lyrics to Predictions
df_test["predicted_label"] = df_test["cleaned_lyrics"].apply(predict_genre)

# Convert Labels from Index to Genre Names
df_test["predicted_genre"] = df_test["predicted_label"].apply(lambda x: label_encoder.classes_[x])
df_test["true_label"] = label_encoder.transform(df_test["genre"])

# Print Classification Report
report = classification_report(df_test["true_label"], df_test["predicted_label"], target_names=label_encoder.classes_)
print("\n📊 Classification Report:\n")
print(report)

# Save Results to CSV for Visualization
df_test.to_csv("test_predictions.csv", index=False)
print("✅ Predictions saved to 'test_predictions.csv'")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

✅ Loaded Test Data! Shape: (5675, 2)

📊 Classification Report:

              precision    recall  f1-score   support

       blues       0.36      0.30      0.32       926
     country       0.52      0.58      0.55      1123
     hip hop       0.97      0.57      0.71       198
        jazz       0.35      0.42      0.38       731
         pop       0.42      0.52      0.46      1398
      reggae       0.50      0.43      0.46       515
        rock       0.47      0.29      0.36       784

    accuracy                           0.44      5675
   macro avg       0.51      0.44      0.46      5675
weighted avg       0.45      0.44      0.44      5675

✅ Predictions saved to 'test_predictions.csv'
